In [32]:
import ROOT
import os
import numpy as np
import pandas as pd

In [ ]:
from math import log10, floor
def round_to_1(x):
    return round(x, -int(floor(log10(abs(x)))))
def round_to_reference(x, y):
    return round(x, -int(floor(log10(y))))

In [21]:
def get_dataframe_reco(filepath="",CookType="TBT",Target="NH3",runMin=-1000,runMax=9999999,currentMax=9999999):
    rcdb = pd.read_csv(filepath+"/rcdb.csv")
    files = []
    validruns=rcdb[(rcdb.CookType==CookType) & (rcdb.Target == Target) & (rcdb.Run>=runMin) & (rcdb.Run<=runMax) & (rcdb.beamCurrent < currentMax)].Run.to_list()
    actuallyTBT=[16408, 16414, 16421, 16424, 16659, 16665, 16671, 16674, 16676, 16679,
16682, 16685, 16688, 16692, 16709, 16710, 16711, 16712, 16713, 16715,
16719, 16723, 16727, 16729, 16730, 16732, 16736, 16742, 16743, 16748,
16752]
    if(CookType=="TBT"):
        validruns+=[r for r in rcdb.Run if (r in actuallyTBT and r>=runMin and r<=runMax)]
    elif(CookType=="HBT"):
        vr=[r for r in validruns if (not r in actuallyTBT)]
        validruns=vr
    for run in validruns:
        files.append(filepath+"/run"+str(run)+".root")
    print("Valid Runs for Analysis -->",np.sort(validruns))
    return ROOT.RDataFrame("tree_postprocess",files)

def get_dataframe_MC(filepath=""):
    return ROOT.RDataFrame("tree_postprocess",filepath+"/merged_run.root")

In [ ]:
def missing_mass_peak(df,df2="",binname="",bins=[]):
    if(binname=="" or bins==[]):
        binname="x"
        bins=[[0,1]]
    else:
        BINS=[]
        for binidx in range(len(bins)-1):
            BINS.append([bins[binidx],bins[binidx+1]])
        bins=BINS
    
    width_per_plot = 600
    height_per_plot = 600
    nPlots = len(bins)
    cols = 4
    rows = int(np.ceil(nPlots/cols))
    if(rows==1):
        cols=nPlots
    
    c = ROOT.TCanvas("c","c",width_per_plot*cols,height_per_plot*rows)
    c.Divide(cols,rows)
    print(cols,rows)
    hr1=[]
    hr2=[]
    for i in range(nPlots):
        hr1.append(df.Filter("{}>{} && {}<{}".format(binname,bins[i][0],binname,bins[i][1])).Histo1D(("h","ep(NH3)#rightarrowe'#pi^{+}#pi^{-}X (TBT);Missing Mass [GeV];",100,0.7,1.3),"Mmiss"))
        if(df2):
            hr2.append(df2.Filter("{}>{} && {}<{}".format(binname,bins[i][0],binname,bins[i][1])).Histo1D(("h2","ep(NH3)#rightarrowe'#pi^{+}#pi^{-}X (TBT);Missing Mass [GeV];",100,0.7,1.3),"Mmiss_p"))
    for j in range(nPlots):
        c.cd(j+1)
        hr1[j].SetLineColor(1)
        if(df2):
            hr2[j].SetLineColor(ROOT.kBlue)
        if(hr1[j].Integral()<10):
            continue
        f1=ROOT.TF1("f1","gaus(0)+pol2(3)",0.75,1.25)
        f1.SetParameters(10,0.938,0.2,1,1,1)
        f1.SetParLimits(0,1,10000)
        f1.SetParLimits(1,0.9,1)
        f1.SetParLimits(2,0.02,1)
        hr1[j].Fit(f1,"SR Q0")
        hr1[j].Draw()
        hr1[j].GetYaxis().SetRangeUser(0,1.25*hr1[j].GetBinContent(hr1[j].GetMaximumBin()))
        if(df2):
            hr2[j].Draw("same")
        #if(f1.GetChisquare()/f1.GetNDF()>10):
        #    continue
        f1.Draw("same")
        latex=ROOT.TLatex()
        latex.SetTextSize(0.03)
        redlatex=ROOT.TLatex()
        redlatex.SetTextColor(ROOT.kRed)
        redlatex.SetTextSize(0.03)
        for i in range(f1.GetNpar()):
            latex.DrawLatexNDC(0.13,0.7-0.03*i,"a{}={}#pm{}".format(i,
                                                       round_to_reference(f1.GetParameter(i),f1.GetParError(i)),
                                                       round_to_1(f1.GetParError(i))))
        latex.DrawLatexNDC(0.13,0.7-0.03*f1.GetNpar()-0.01,"#chi^{2}/ndf="+"{:.2f}/{}".format(f1.GetChisquare(),f1.GetNDF())+"={:.2f}".format(f1.GetChisquare()/f1.GetNDF()))
        
        latex.DrawLatexNDC(0.5,0.14,"N within 3#sigma #rightarrow {:.0f}".format(hr1[j].Integral(hr1[j].FindBin(f1.GetParameter(1)-3*f1.GetParameter(2)),
                                                                         hr1[j].FindBin(f1.GetParameter(1)+3*f1.GetParameter(2)))))
        
        latex.DrawLatexNDC(0.13,0.8,"a_{0}exp(- #frac{(x-a_{1})^{2}}{2a_{2}^{2}}) + a_{3} + a_{4}x + a_{5}x^{2}")
        if(nPlots>1):
            redlatex.DrawLatexNDC(0.65,0.8,"{:.3f} < {} < {:.3f}".format(bins[j][0],binname,bins[j][1]))
    
        if(j==0):
            legend=ROOT.TLegend(0.5,0.26,0.85,0.43)
            legend.AddEntry(hr1[j].GetValue(),"All e'#pi^{+}pi^{-} final states","l")
            legend.AddEntry(f1,"Gaussian+Poly fit","l")
            if(df2):
                legend.AddEntry(hr2[j].GetValue(),"All e'#pi^{+}#pi^{-}p' final states","l")
            legend.Draw("same")
        ROOT.gStyle.SetOptStat(0)
        
    c.SaveAs("test_{}.png".format(binname))